In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import genextreme
import os
import sys

operating_system = 'mac'

if operating_system == 'win':
    os.chdir('C:/Users/fabau/OneDrive/Documents/GitHub/master-project-cleaned/')
elif operating_system == 'curnagl':
    os.chdir('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/')
else:
    os.chdir('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/')

util_perso = os.path.abspath('util/gev')
sys.path.append(util_perso)
util_perso = os.path.abspath('util/processing')
sys.path.append(util_perso)

import time_series
from data_processing import depickle

In [ ]:
final = '_final'

model_u = depickle(f'analysis_loadings_selected/u_gust{final}/best_linear_max_model.pkl')

pcaall = depickle(f'analysis_loadings_selected/PCA_fred{final}/pcaall.pkl')

pcsall_train = depickle(f'analysis_loadings_selected/PCA_fred{final}/pcsall_train.pkl')
pcsall_test = depickle(f'analysis_loadings_selected/PCA_fred{final}/pcsall_test.pkl')
pcsall_valid = depickle(f'analysis_loadings_selected/PCA_fred{final}/pcsall_valid.pkl')

# the storm number linked to the index
train_number = pd.read_csv('ml_scripts/new_feature_selection/seed_42/X_train_ts_all.csv')['storm_index']
valid_number = pd.read_csv('ml_scripts/new_feature_selection/seed_42/X_validation_ts_all.csv')['storm_index']
train_valid = pd.concat([train_number, valid_number], axis=0).reset_index(drop=True)

test_number = pd.read_csv('ml_scripts/new_feature_selection/seed_42/X_test_ts_all.csv')['storm_index']

#rmses_u_all = []
rmses_u_all = []
for a in range(7):

    train_set_vivian =  depickle('analysis_loadings_selected/PCA_fred_final/train_sets.pkl')[0]
    pcs0_vivian = pcsall_train[0] #pd.read_csv(f'analysis_loadings_selected/PCA_fred_final/yn/ytrain_split_{a}.pkl')#pcsall_train[a]
    pcs0_lothar = pcsall_test[0]

    # loadings on the train set, validation set and test set
    train_sets = depickle('analysis_loadings_selected/PCA_fred_final/train_sets.pkl')[a]
    val_sets = depickle('analysis_loadings_selected/PCA_fred_final/val_sets.pkl')[a]

    pcs0_train = pcsall_train[a] #pd.read_csv(f'analysis_loadings_selected/PCA_fred_final/yn/ytrain_split_{a}.pkl')#pcsall_train[a]
    pcs0_valid = pcsall_valid[a]# pd.read_csv(f'analysis_loadings_selected/PCA_fred_final/yn/yvalid_split_{a}.pkl')#pcsall_valid[a]
    pcs0_test = pcsall_test[a]

    output_type = ['max']

    for out in output_type:
        if out == 'max':
            folder = f'u_gust{final}'
        else:
            folder = f'z_gust{final}'

        var_selected = features = pd.DataFrame(depickle(f'analysis_loadings_selected/{folder}/best_linear_{out}_feature.pkl'),
                            columns = ['var_name', 'PC'])
        var_selected ['var_name_pca'] = var_selected ['var_name']+ '_PC'+ var_selected ['PC'].astype(str)

        loading_train = []
        loading_valid = []
        loading_test = []

        loading_vivian = []
        loading_lothar = []
    
        eigenvectors = {}

        for var in var_selected['var_name']:

            pcn = var_selected[var_selected['var_name']==var]['PC'].values[0]
            #print(var)

            # combine all loadings for the selected variables in one dataframe with the variable names as index and the storm number as columns
            temp_loading_allpcn_train = pcs0_train[var][:]
            temp_loading_seppcn_train = temp_loading_allpcn_train[:,pcn]
            loading_train.append(temp_loading_seppcn_train)

            temp_loading_allpcn_valid = pcs0_valid[var][:]
            temp_loading_seppcn_valid = temp_loading_allpcn_valid[:,pcn]
            loading_valid.append(temp_loading_seppcn_valid)

            temp_loading_allpcn_test = pcs0_test[var][:]
            temp_loading_seppcn_test = temp_loading_allpcn_test[:,pcn]
            loading_test.append(temp_loading_seppcn_test)

            temp_loading_allpcn_vivian = pcs0_vivian[var][:]
            temp_loading_seppcn_vivian = temp_loading_allpcn_vivian[:,pcn]
            loading_vivian.append(temp_loading_seppcn_vivian)

            temp_loading_allpcn_lothar = pcs0_lothar[var][:]
            temp_loading_seppcn_lothar = temp_loading_allpcn_lothar[:,pcn]
            loading_lothar.append(temp_loading_seppcn_lothar)

            # extract the scores from pcaall
            scores = {}  # dictionary to store dataframes

            #if  var.startswith('10m_'):
            #    continue
            #if var.startswith('mean_sea_level_pressure_min'):
            #    continue
            #loadings[var] = pd.read_csv(f'data/PCA/PCA_loadings_1000/{var}.csv')#.drop(columns=['variable'])
            eigenvectors[var] = pcaall[var].components_[pcn,:] # or vector
            #explained_variance_10 = pcaall[var].explained_variance_[:10] # or value
            #loadings[var] = pd.DataFrame(loadings_10, columns=[f'PCA_{i+1}' for i in range(10)])
            #scores[var] = pd.read_csv(f'data/PCA/PCA_scores_1000/{var}.csv').drop(columns=['variable'])

    loading_train = np.array(loading_train)
    loading_train = pd.DataFrame(loading_train, 
                                index=var_selected['var_name_pca'],
                                columns=train_sets)
    loading_valid = np.array(loading_valid)
    loading_valid = pd.DataFrame(loading_valid, 
                                index=var_selected['var_name_pca'],
                                columns=val_sets)
    loading_test = np.array(loading_test)
    loading_test = pd.DataFrame(loading_test, 
                                index=var_selected['var_name_pca'],
                                columns=test_number)
    loading_test_clean = loading_test.T
    loading_test_clean = loading_test_clean[loading_test_clean.iloc[:, 0] != 'storm_index']

    loadings_train_valid = pd.concat([loading_train, loading_valid], axis=1)

    loadings_train_valid = loadings_train_valid.rename(columns=train_valid)

    loadings_all = pd.concat([loadings_train_valid, loading_test], axis=1).T

    # storm specific loadings
    loading_vivian = np.array(loading_vivian)
    loading_vivian = pd.DataFrame(loading_vivian, 
                                index=var_selected['var_name_pca'],
                                columns=train_set_vivian)
    
    loading_lothar = np.array(loading_lothar)
    loading_lothar = pd.DataFrame(loading_lothar,
                                index=var_selected['var_name_pca'],
                                columns=test_number)

    #prediction_z_train = model_u.predict(loading_train.T)
    prediction_u_train = model_u.predict(loading_train.T)
    #prediction_z_valid = model_z.predict(loading_valid.T)
    prediction_u_valid = model_u.predict(loading_valid.T)
    #prediction_z_test = model_z.predict(loading_test_clean)
    prediction_u_test = model_u.predict(loading_test_clean)

    prediction_vivian = model_u.predict(loading_vivian.T)
    prediction_lothar = model_u.predict(loading_lothar.T)

    #prediction_z_train_valid = model_z.predict(loadings_train_valid.T)
    prediction_u_train_valid = model_u.predict(loadings_train_valid.T)
    true_winds = pd.read_csv('data/climatology_dm_winter_per_cluster/EVENT_max/max_event_dm_combined.csv', index_col=0)
    
    #rmses_z = []
    rmses_u = []

    # for i in range(len(15)):  # Assuming all lists/arrays have the same length
    #     prediction_z_train = prediction_z_train[i]  # Access individual elements
    #     #prediction_u_train = prediction_u_train[i], prediction_u_valid[i], prediction_u_test[i])  # Access individual elements
    #     prediction_z_valid = prediction_z_valid[i]  # Access individual elements
    #     #prediction_u_valid = prediction_u_valid[i]  # Access individual elements
    #     prediction_z_test = prediction_z_test[i]  # Access individual elements
    #     #prediction_u_test = prediction_u_test[i]  # Access individual elements

    true_winds_Vivian = true_winds[true_winds['storm_name'] == 'VIVIAN']
    true_winds_Lothar = true_winds[true_winds['storm_name'] == 'LOTHAR']
    #for n in range(3):
    n = 3
    n_lothar = 2

    #cluster_z = prediction_z[n,:]
    cluster_u = prediction_vivian[n,:]
    cluster_u_lothar = prediction_lothar[n_lothar,:]

    true_winds_mean = np.mean(true_winds.drop(columns='storm_name'), axis=0)

    prediction_u_train_mean = np.mean(prediction_u_train, axis=0)
    prediction_u_valid_mean = np.mean(prediction_u_valid, axis=0)
    prediction_u_test_mean = np.mean(prediction_u_test, axis=0)

    #prediction_u_mean = np.mean(prediction_u, axis=0)

    cluster_u = pd.DataFrame(cluster_u, columns=['cluster_u'])
    cluster_u_lothar = pd.DataFrame(cluster_u_lothar, columns=['cluster_u_lothar'])

    '''max_cluster_z = np.max(cluster_z)
    min_cluster_z = np.min(cluster_z)

    max_cluster_u = np.max(cluster_u)
    min_cluster_u = np.min(cluster_u)'''

    # cluster_z_3 = cluster_z[3]
    # cluster_u_3 = cluster_u[3]

    #cluster_u_wind = []

    # cluster_z_train_wind_mean = []
    # cluster_z_valid_wind_mean = []
    # cluster_z_test_wind_mean = []

    # for i in range(15):

    #     #cluster_z_3 = cluster_z[i]

    #     cluster_train_mean_z = prediction_u_train_mean[i]
    #     cluster_valid_mean_z = prediction_u_valid_mean[i]
    #     cluster_test_mean_z = prediction_u_test_mean[i]


    #     # for cluster n°3
    #     param = pd.read_csv(f'data/climatology_dm_winter_per_cluster/GEV_parameters/GEV_parameters_cluster_{n+1}.csv', index_col=0)
    #     shape_param= param.iloc[0].values
    #     loc_param = param.iloc[1].values
    #     scale_param = param.iloc[2].values

    #     #max_cluster_z_perc = 1-np.exp(-cluster_z_3)
    #     cluster_train_mean_z_perc = 1-np.exp(-cluster_train_mean_z)
    #     cluster_valid_mean_z_perc = 1-np.exp(-cluster_valid_mean_z)
    #     cluster_test_mean_z_perc = 1-np.exp(-cluster_test_mean_z)
    #     # Percentile (e.g., 90%)
    #     #percentile = max_cluster_z_perc

    #     # Calculate wind speed at the given percentile
    #     #wind_speed_z = genextreme.ppf(percentile, c=shape_param, loc=loc_param, scale=scale_param)
    #     wind_speed_train_z_mean = genextreme.ppf(cluster_train_mean_z_perc, c=shape_param, loc=loc_param, scale=scale_param)
    #     wind_speed_valid_z_mean = genextreme.ppf(cluster_valid_mean_z_perc, c=shape_param, loc=loc_param, scale=scale_param)
    #     wind_speed_test_z_mean = genextreme.ppf(cluster_test_mean_z_perc, c=shape_param, loc=loc_param, scale=scale_param)

    #     #cluster_z_wind.append(wind_speed_z)
    #     cluster_z_train_wind_mean.append(wind_speed_train_z_mean)
    #     cluster_z_valid_wind_mean.append(wind_speed_valid_z_mean)
    #     cluster_z_test_wind_mean.append(wind_speed_test_z_mean)

    #cluster_u_wind = pd.DataFrame(cluster_u_wind, columns=['cluster_u_wind'])
    cluster_u_train_wind_mean = pd.DataFrame(prediction_u_train_mean, columns=['cluster_u_train_wind_mean'])
    cluster_u_valid_wind_mean = pd.DataFrame(prediction_u_valid_mean, columns=['cluster_u_valid_wind_mean'])
    cluster_u_test_wind_mean = pd.DataFrame(prediction_u_test_mean, columns=['cluster_u_test_wind_mean'])

    true_winds_Vivian = true_winds_Vivian.T[1:].reset_index(drop=True)
    true_winds_Vivian = true_winds_Vivian.rename(columns={5: 'true_winds_Vivian'})

    true_winds_Lothar = true_winds_Lothar.T[1:].reset_index(drop=True)
    true_winds_Lothar = true_winds_Lothar.rename(columns={38: 'true_winds_Lothar'})

    true_winds_mean = pd.DataFrame(true_winds_mean, columns=['true_winds_mean']).reset_index(drop=True)
    #prediction_u_mean = pd.DataFrame(prediction_u_mean, columns=['prediction_u_mean'])

    winds_vivian = pd.concat([true_winds_Vivian, cluster_u], axis=1)
    winds_vivian.to_csv('analysis_loadings_selected/figures/winds_vivian_u.csv')

    winds_lothar = pd.concat([true_winds_Lothar, cluster_u_lothar], axis=1)
    winds_lothar.to_csv('analysis_loadings_selected/figures/winds_lothar_u.csv')

    winds_cluster_train_mean = pd.concat([true_winds_mean, cluster_u_train_wind_mean], axis=1) # prediction_u_mean
    winds_cluster_valid_mean = pd.concat([true_winds_mean, cluster_u_valid_wind_mean], axis=1) # prediction_u_mean 
    winds_cluster_test_mean = pd.concat([true_winds_mean, cluster_u_test_wind_mean], axis=1) # prediction_u_mean

    winds_error_train_u = winds_cluster_train_mean['true_winds_mean'] - winds_cluster_train_mean['cluster_u_train_wind_mean']
    winds_error_valid_u = winds_cluster_valid_mean['true_winds_mean'] - winds_cluster_valid_mean['cluster_u_valid_wind_mean']
    winds_error_test_u = winds_cluster_test_mean['true_winds_mean'] - winds_cluster_test_mean['cluster_u_test_wind_mean']

    #winds_error_u = winds_cluster_mean['true_winds_mean'] - winds_cluster_mean['prediction_u_mean']

    rmse_u_train = np.sqrt(np.mean(winds_error_train_u**2))
    rmse_u_valid = np.sqrt(np.mean(winds_error_valid_u**2))
    rmse_u_test = np.sqrt(np.mean(winds_error_test_u**2))

    #rmse_u = np.sqrt(np.mean(winds_error_u**2))

    #print(rmse_z)
    #print(rmse_u)

    rmses_u.append(rmse_u_train)
    print(rmse_u_train, 'train')
    rmses_u.append(rmse_u_valid)
    print(rmse_u_valid, 'valid')
    rmses_u.append(rmse_u_test)
    print(rmse_u_test, 'test')

    print(rmses_u)

    #rmses_u.append(rmse_u)

#rmses_u_all.append(rmses_u)
    rmses_u_all.append(rmses_u)

#winds_vivian = pd.concat([true_winds_Vivian, cluster_z_wind], axis=1) # cluster_u


#rmses_z_all_averaged = np.mean(rmses_z_all, axis=0)
rmses_u_all_averaged = np.mean(rmses_u_all, axis=0)

#pd.DataFrame(rmses_z_all_averaged).to_csv('analysis_loadings_selected/figures/rmse_z_all.csv')
pd.DataFrame(rmses_u_all_averaged).to_csv('analysis_loadings_selected/figures/rmse_u_all.csv')
'''
plt.plot(rmses_z_all_averaged, label='Z_gust', marker='o', linestyle='None')
#plt.plot(rmses_u_all_averaged, label='U_gust', marker='s', linestyle='None')
plt.xticks(range(3), ['train', 'validation', 'test'])
plt.xlabel('Dataset')
plt.ylabel('RMSE in m/s')
plt.grid()
plt.legend()
#plt.savefig('analysis_loadings_selected/figures/rmse_z_u_gust.png', dpi=300)
'''

/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


0.3291882530691065 train
1.6871164314924083 valid
4.174823860300746 test
[0.3291882530691065, 1.6871164314924083, 4.174823860300746]
0.8626883312784542 train
2.027217922270463 valid
3.7032676569426 test
[0.8626883312784542, 2.027217922270463, 3.7032676569426]
0.6568383885954976 train
1.4466826445973477 valid
3.984300717135744 test
[0.6568383885954976, 1.4466826445973477, 3.984300717135744]
1.2434957545494063 train
1.114004798504074 valid
3.786855439671143 test
[1.2434957545494063, 1.114004798504074, 3.786855439671143]
0.7475709706741228 train
2.0615189437123127 valid
3.9706468485022683 test
[0.7475709706741228, 2.0615189437123127, 3.9706468485022683]
0.9216536835012549 train
1.2386099491246823 valid
4.039098882148495 test
[0.9216536835012549, 1.2386099491246823, 4.039098882148495]
0.6134415435195443 train
1.221375460166585 valid
3.971516779085708 test
[0.6134415435195443, 1.221375460166585, 3.971516779085708]


/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.w

"\nplt.plot(rmses_z_all_averaged, label='Z_gust', marker='o', linestyle='None')\n#plt.plot(rmses_u_all_averaged, label='U_gust', marker='s', linestyle='None')\nplt.xticks(range(3), ['train', 'validation', 'test'])\nplt.xlabel('Dataset')\nplt.ylabel('RMSE in m/s')\nplt.grid()\nplt.legend()\n#plt.savefig('analysis_loadings_selected/figures/rmse_z_u_gust.png', dpi=300)\n"